### Before removing movies that were getting recommended to new users

In [ ]:
import pandas as pd

# Load the data from the provided CSV files
kafka_data = pd.read_csv('kafka_data.csv')
record_data = pd.read_csv('record.csv',usecols=range(21))

# Extract movie titles from Kafka data by parsing the request URLs
kafka_data['movie_title'] = kafka_data['request'].str.extract(r'/m/(.+?)/')
kafka_movie_counts = kafka_data['movie_title'].value_counts()

record_flat = pd.DataFrame(record_data.stack()).reset_index()
record_flat.columns = ['user_index', 'movie_index', 'movie_title']
record_recommendation_counts = record_flat['movie_title'].value_counts()

# Analyze the distribution of recommendations
top_20_recommended = record_recommendation_counts.head(20)
total_recommendations = record_recommendation_counts.sum()
top_20_percentage = top_20_recommended.sum() / total_recommendations * 100

# Output the analysis
print("Kafka Data - Unique Movies Statistical Summary:")
print(kafka_movie_counts.describe())

print("\nTop 20 Recommended Movies in Record Data:")
print(top_20_recommended)

print(f"\nTop 20 movies as a percentage of all recommendations: {top_20_percentage:.2f}%")


Kafka Data - Unique Movies Statistical Summary:
count    9084.000000
mean       36.591810
std       294.147185
min         3.000000
25%         6.000000
50%         9.000000
75%        15.000000
max      7524.000000
Name: count, dtype: float64

Top 20 Recommended Movies in Record Data:
movie_title
the+shawshank+redemption+1994                       38011
senna+2010                                          37881
spies+1928                                          37851
spontaneous+combustion+1990                         37844
stanley+kubrick+a+life+in+pictures+2001             37839
matthews+days+1968                                  37766
harry+potter+and+the+deathly+hallows+part+1+2010    37729
raiders+of+the+lost+ark+1981                        37697
smile+1975                                          37696
inception+2010                                      37692
spin+2007                                           37687
harry+potter+and+the+deathly+hallows+part+2+2011    37684
divor

### After removing movies that were being served to new users

In [12]:
import pandas as pd

column_names = ['user_id'] + [f'movie_{i}' for i in range(1, 21)]
record_data = pd.read_csv('record.csv', header=None, names=column_names, usecols=range(21))

# Flatten the record data to make it easier to work with
record_flat = pd.melt(record_data, id_vars='user_id', value_vars=[f'movie_{i}' for i in range(1, 21)])
record_flat.columns = ['user_id', 'movie_index', 'movie_title']

# Exclude pseudo-randomly served movies from the record data
pseudo_movie_ids = ['10+things+i+hate+about+you+1999', 'spriggan+1998', 'stargate+sg-1+children+of+the+gods+-+final+cut+2009', 'stargate+continuum+2008', 'star+wars+episode+iii+-+revenge+of+the+sith+2005', 'star+wars+episode+i+-+the+phantom+menace+1999', 'star+wars+1977', 'stanley+kubrick+a+life+in+pictures+2001', 'standing+up+2013', 'standing+in+the+shadows+of+motown+2002', 'stand+up+and+fight+1939', 'stalag+17+1953', 'spring+forward+1999', 'spread+2009', 'senna+2010', 'spontaneous+combustion+1990','spinning+plates+2013', 'spin+2007', 'spies+1928','the+matrix+1999', 'life+as+a+house+2001',"the+shawshank+redemption+1994","the+dark+knight+2008","inception+2010","raiders+of+the+lost+ark+1981","my+neighbor+totoro+1988","forrest+gump+1994","harry+potter+and+the+deathly+hallows+part+2+2011","monty+python+and+the+holy+grail+1975","the+lord+of+the+rings+the+return+of+the+king+2003","spirited+away+2001","the+godfather+1972","the+lord+of+the+rings+the+fellowship+of+the+ring+2001","fight+club+1999","nausica+of+the+valley+of+the+wind+1984","the+green+mile+1999","toy+story+1995","goodfellas+1990","the+dark+knight+rises+2012","seven+samurai+1954", 'the+big+clock+1948', 'matthews+days+1968', 'the+day+of+the+crows+2012', 'mansfield+park+1999', 'kes+1969', 'beauty+and+the+beast+1991', 'red+2008', 'the+dirty+dozen+1967', 'smile+1975', 'interstellar+2014', 'never+say+never+again+1983', 'good+night_+and+good+luck.+2005', 'steal+this+film+2006', 'scarface+1983', 'harry+potter+and+the+deathly+hallows+part+1+2010', 'the+little+mermaid+1989', 'you+and+i+2006', 'the+chronicles+of+narnia+prince+caspian+2008','crumb+1994', 'divorce+american+style+1967', 'cool+hand+luke+1967', 'chappie+2015', 'walle+2008', 'where+the+red+fern+grows+1974', '24+exposures+2013', 'rain+man+1988', 'hugo+2011', 'lost+in+la+mancha+2002', 'sergeant+york+1941', 'westward+the+women+1951']
record_flat = record_flat[~record_flat['movie_title'].isin(pseudo_movie_ids)]

# Analyze the distribution of recommendations
record_recommendation_counts = record_flat['movie_title'].value_counts()
top_20_recommended = record_recommendation_counts.head(20)
total_recommendations = record_recommendation_counts.sum()
top_20_percentage = top_20_recommended.sum() / total_recommendations * 100

# Output the analysis
print("Top 20 Recommended Movies in Record Data (Excluding Pseudo-Randomly Served Movies):")
print(top_20_recommended)

print(f"\nTop 20 movies as a percentage of all recommendations (excluding pseudo-randomly served movies): {top_20_percentage:.2f}%")


Top 20 Recommended Movies in Record Data (Excluding Pseudo-Randomly Served Movies):
movie_title
('the+shawshank+redemption+1994', 0.8128104)           2
('my+best+friend+2006', 0.72639185)                    2
('back+to+the+future+1985', 0.81434804)                2
('the+secret+world+of+arrietty+2010', 0.7579194)       2
('gattaca+1997', 0.8469027)                            2
('brave+2012', 0.7451509)                              2
('senna+2010', 0.89109594)                             2
('steamboat+bill_+jr.+1928', 0.7421252)                2
('himalaya+1999', 0.7409592)                           2
('sparkle+2012', 0.75481737)                           2
('the+matrix+1999', 0.7794365)                         2
('sparkle+2012', 0.85248643)                           2
('the+kingdom+2007', 0.7728366)                        2
('the+shawshank+redemption+1994', 0.9016538)           2
('brave+2012', 0.76727796)                             2
('monsters_+inc.+2001', 0.84533775)              

### Number of Unique Users

In [ ]:
import pandas as pd

# Load the data
record_data = pd.read_csv('record.csv', usecols=range(21))

# Calculate total number of rows
total_rows = len(record_data)

# The first column contains the user IDs, calculate unique customers
unique_customers = record_data.iloc[:, 0].nunique()

# Output the results
print(f"Total rows in record.csv: {total_rows}")
print(f"Unique customers: {unique_customers}")

Total rows in record.csv: 131482
Unique customers: 126024
